In [1]:
#!pip install pandas openpyxl

In [2]:
import pandas as pd
df500_closingPrice = pd.read_excel(r"中证500股指期货收盘价(主力合约)2022-12-31.xlsx", engine='openpyxl')


In [3]:
df500_closingPrice.head()

,指标名称,中证500股指期货:收盘价(主力合约)
0,频率,日
1,单位,元/张
2,指标ID,M004349674
3,2022-12-30,5870.8
4,2022-12-29,5869.4


In [4]:
#delete first three rows 
df500_closingPrice = df500_closingPrice.iloc[3:]
#change feature name to "date" and "close"
df500_closingPrice.columns = ["date", "close"]
df500_closingPrice.head()

,date,close
3,2022-12-30,5870.8
4,2022-12-29,5869.4
5,2022-12-28,5882.4
6,2022-12-27,5897.6
7,2022-12-26,5860.6


In [5]:
df500_closingPrice['date']=pd.to_datetime(df500_closingPrice['date'])
df500_closingPrice.set_index("date", inplace=True)
weekly_groups=df500_closingPrice.groupby(pd.Grouper(freq='W'))
weekly_closingPrice=weekly_groups.mean()
weekly_closingPrice


,close
date,
2015-04-19,7733.00
2015-04-26,8109.04
2015-05-03,8283.80
2015-05-10,8123.12
2015-05-17,8603.24
...,...
2022-12-04,6155.48
2022-12-11,6204.36
2022-12-18,6123.32


In [6]:
#add index week1 week2 week3...
weekly_closingPrice['week'] = range(1, len(weekly_closingPrice) + 1)
weekly_closingPrice


,close,week
date,,
2015-04-19,7733.00,1
2015-04-26,8109.04,2
2015-05-03,8283.80,3
2015-05-10,8123.12,4
2015-05-17,8603.24,5
...,...,...
2022-12-04,6155.48,399
2022-12-11,6204.36,400
2022-12-18,6123.32,401


In [7]:
# make close column value natural log of close
import numpy as np
weekly_closingPrice['close'] = np.log(weekly_closingPrice['close'])
weekly_closingPrice

,close,week
date,,
2015-04-19,8.953252,1
2015-04-26,9.000735,2
2015-05-03,9.022057,3
2015-05-10,9.002470,4
2015-05-17,9.059894,5
...,...,...
2022-12-04,8.725098,399
2022-12-11,8.733008,400
2022-12-18,8.719860,401


In [8]:
#check missing value
weekly_closingPrice.isnull().sum()
#find out missing value
weekly_closingPrice[weekly_closingPrice.isnull().values==True]


,close,week
date,,
2016-02-14,NaN,44
2016-10-09,NaN,78
2017-10-08,NaN,130
2018-10-07,NaN,182
2019-02-10,NaN,200
2020-02-02,NaN,251
2022-02-06,NaN,356
2022-10-09,NaN,391


In [9]:
#drop missing value
weekly_closingPrice.dropna(inplace=True)


In [10]:
weekly_closingPrice.set_index("week", inplace=True)
weekly_closingPrice

,close
week,
1,8.953252
2,9.000735
3,9.022057
4,9.002470
5,9.059894
...,...
399,8.725098
400,8.733008
401,8.719860


In [11]:
#diff function
weekly_closingPrice['closingPrice'] = weekly_closingPrice['close'].diff(periods=1)
weekly_closingPrice

,close,closingPrice
week,,
1,8.953252,NaN
2,9.000735,0.047483
3,9.022057,0.021322
4,9.002470,-0.019587
5,9.059894,0.057425
...,...,...
399,8.725098,0.002551
400,8.733008,0.007910
401,8.719860,-0.013148


In [12]:
#delete closing_price column
del weekly_closingPrice['close']
weekly_closingPrice

,closingPrice
week,
1,NaN
2,0.047483
3,0.021322
4,-0.019587
5,0.057425
...,...
399,0.002551
400,0.007910
401,-0.013148


In [13]:
#to csv
weekly_closingPrice.to_csv(r'500_Future_return_weekly.csv', index = False, header=True)
